In [1]:
import keras
from keras.layers import Lambda, Flatten, Dense, Input
from keras.models import Sequential
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np

In [4]:
path = 'D:\\fake-vs-real\\real_vs_fake\\real-vs-fake\\'
image_gen = ImageDataGenerator(rescale=1./255.)
batch_size = 64

train_flow = image_gen.flow_from_directory(
    path + 'train/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

valid_flow = image_gen.flow_from_directory(
    path + 'valid/',
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary'
)

test_flow = image_gen.flow_from_directory(
    path + 'test/',
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,
    class_mode='binary'
)

Found 100000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.


In [6]:
# Add preprocessing layer to the front of VGG
vgg = VGG16(input_shape = (224, 224, 3), weights = 'imagenet', include_top = False)

# Don't train existing weights
for layer in vgg.layers:
    layer.trainable = False

In [7]:
# Our layers, can add more if we want
x = Flatten()(vgg.output)
# x = Dense(1000, actiavtion = 'relu')(x)
prediction = Dense(units = 1, activation = 'sigmoid')(x)

In [8]:
# Create a model object
classifier = Model(inputs = vgg.input, outputs = prediction)

In [9]:
classifier.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)      

In [10]:
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
train_steps = 100000//batch_size
valid_steps = 20000//batch_size

history = classifier.fit_generator(
    train_flow,
    epochs = 5,
    steps_per_epoch = train_steps,
    validation_data = valid_flow,
    validation_steps = valid_steps)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
 365/1562 [======>.......................] - ETA: 4:40:48 - loss: 0.4890 - accuracy: 0.7626